In [2]:
import requests
import numpy as np
import re
import json
import time
import pandas as pd
from collections import deque
from bs4 import BeautifulSoup

In [2]:
# Application name 	collaborative-filtering
# API key 	b2192064c900b89ae6627a1a9871ec37
# Shared secret 	20702a1e39bcac00be717a0b93df1dd5
# Registered to 	aidan_bbq

## Scrape usernames via popular artists

In [2]:
API_KEY=None
USER_AGENT=None
PASSWORD=None
with open("./secrets.txt", "r") as s:
    lines = s.readlines()
    API_KEY = lines[0].strip("\n")
    USER_AGENT = lines[1].strip("\n")
    PASSWORD = lines[2].strip("\n")

In [3]:
def lastfm_get(payload):
    # define headers and URL
    headers = {'user-agent': USER_AGENT}
    url = 'https://ws.audioscrobbler.com/2.0/'

    # Add API key and format to the payload
    payload['api_key'] = API_KEY
    payload['format'] = 'json'

    response = requests.get(url, headers=headers, params=payload)
    return response

In [4]:
def getTopArtists(n=50):
    payload = {
        "method": "chart.gettopartists",
        "limit": n
    }
    r = lastfm_get(payload)
    artists = r.json()['artists']["artist"]
    return [a['name'] for a in artists]

In [5]:
def getUserInfo(user):
    payload = {
        "method": "user.getinfo",
        "user": user
    }
    r = lastfm_get(payload)
    j = r.json()["user"]
    return [j["name"], j["country"], j["playcount"], j["registered"]["unixtime"], j["gender"]]

In [6]:
def getTopArtists_user(user, client, file, userid, artistids, n=50):
    periods = ["overall"]#, "1month", "12month"]
    output=[]
    for p in periods:
        payload = {
            "method": "user.gettopartists",
            "user": user,
            "period": p,
            "limit": n
        }
        r = lastfm_get(payload)
        j = r.json()
        if "topartists" in j and "artist" in j["topartists"]:
            artists = r.json()['topartists']["artist"]
            for a in artists:
                if a['name'] not in artistids:
                    artistids.append(a['name'])
                    f.write(f"{userid},{len(artistids)-1},{int(a['playcount'])}\n")
                else:
                    f.write(f"{userid},{list(artistids).index(a['name'])},{int(a['playcount'])}\n")

## Generate user listening data

In [ ]:
start = time.perf_counter()
client = requests.Session()

users = pd.read_csv("./udata/usernames.csv", header=None)[0].to_numpy()
#     artistids = pd.read_csv("./udata/artists.csv", header=None)[0].to_numpy()
artists = []

ftop = open("./udata/10000users_200listens.csv", "a", encoding="utf-8")
faid = open('./udata/artists.csv', "a", encoding="utf-8")

uid=0
while uid < 10000:
#         print(f"fetching {userids[uid]} w new uid {uid}")
    payload = {
        "method": "user.gettopartists",
        "user": users[uid],
        "period": "overall",
        "limit": 200
    }
    try:
        r = lastfm_get(payload)
        j = r.json()
        if "topartists" in j and "artist" in j["topartists"]:
            listens = r.json()['topartists']["artist"]
            for a in listens:
                if a['name'] not in artists:
                    artists = np.append(artists, a['name'])
                    faid.write(f"{a['name']}\n")
                ftop.write(f"{uid},{np.where(artists == a['name'])[0][0]},{int(a['playcount'])}\n")
    except Exception as e:
        print(e)
        uid -= 1
    uid += 1
ftop.close()
faid.close()
print(time.perf_counter() - start)

## Scrape for users

In [6]:
def getneighbors(username):
    url = f"https://www.last.fm/user/{username}/neighbours"
    resp = requests.get(url)    
    if resp.status_code==200:
        soup = BeautifulSoup(resp.text, 'html.parser')
        us = soup.find("section", {"class": "neighbours-items-section"})
        if us is not None:
            us = us.findAll("a", {"class": "user-list-link link-block-target"})
            return np.asarray([u.text for u in us])
    return np.array([])

In [64]:
def openLastfmClient():
    payload = payload = {
        "username_or_email": USER_AGENT,
        "password": PASSWORD
    }
    loginurl = "https://www.last.fm/login"
    client = requests.Session()
    client.get(loginurl)
    if 'csrftoken' in client.cookies:
        # Django 1.6 and up
        csrftoken = client.cookies['csrftoken']
    else:
        # older versions
        csrftoken = client.cookies['csrf']
    payload["csrfmiddlewaretoken"] = csrftoken
    client.headers.update({'referer': loginurl})
    client.post(loginurl, data=payload)
    return client

In [65]:
def getlisteners(artist, client, pages=[1,2,3]):
    users = []
    for p in pages:
        url = f"https://www.last.fm/music/{artist}/+listeners?page={p}"
        resp = client.get(url)
        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'html.parser')
            us = soup.findAll("h3", {"class": "top-listeners-item-name"})
            users += [u.find("a").text for u in us]
    return users

In [67]:
def crawlTops():
    client = openLastfmClient()    
    artists  = getTopArtists(n=1000)
    my_artists = np.array(getTopArtists_user("aidan_bbq"))
    explore = my_artists[~np.isin(my_artists,artists)]
    with open("./output/toplisteners_me.csv", "a", encoding="utf-8") as f:
        for a in explore:
            print(f"on artist: {a}")
            [f.write(f"{a},{u}\n") for u in getlisteners(a,client)]

## Generate user demographics

In [85]:
with open("./output/users_unique.csv", "r", encoding="utf-8") as f:
    with open("./output/users_info.csv", "w", encoding="utf-8") as fu:
        for user in f.readlines():
            u = getUserInfo(user)
            fu.write(f"{u[0]},{u[1]},{u[2]},{u[3]},{u[4]}\n")

KeyboardInterrupt: 

## Scrape last.fm via user neighbors

In [ ]:
datafile = "./output/toplisteners_me.csv"
with open(datafile, 'r', encoding="utf-8") as f:
    lines = [[node for node in re.split('[,](?!\s)',edge.strip('\n'))[:2]] for edge in f.readlines() if edge[0][0] != '#']
    topls = pd.DataFrame(lines, columns = ["artist", "user"])

In [10]:
userheads = topls.user
searchstack = deque(userheads)
userset = set(userheads)
explored = np.array([]) 
useredges = None # to make graph of users

In [11]:
edges = None
datafile = "./output/both-neighs.csv"
with open(datafile, 'r') as f:
    lines = [[node for node in re.split(',',edge.strip('\n'))[:2]] for edge in f.readlines() if edge[0][0] != '#']
    edges = np.array(lines)

In [12]:
tot_userset = set(edges.flatten())
tot_explored = np.array(set(edges[:,0]))
explored = tot_explored

In [13]:
newf = open("./output/new-neighs.csv", "a", encoding="utf-8")
totf = open("./output/both-neighs.csv", "a", encoding="utf-8")

In [14]:
numnew10 = deque([11]) # number of new users added in the past 10 iterations
while sum(numnew10) > 10  and len(searchstack) > 0:
    curruser = searchstack.popleft()
    if np.isin(curruser, explored):
        continue
    explored = np.append(explored, curruser)

    neighs = getneighbors(curruser)
    prevlen = len(userset)
    userset = userset.union(neighs)
    
    # stoppage conditions
    numnew10.append(len(userset) - prevlen)
    if len(numnew10) > 10:
        numnew10.popleft()
        
    # keeping track of overall additions
    tot_prevlen = len(tot_userset)
    tot_userset = tot_userset.union(userset)
    print(f"explored {curruser}, found {numnew10[-1]} new users, {len(tot_userset) - tot_prevlen} totally new, {len(tot_userset)} total")
    

    edges = np.asarray([np.repeat(curruser, len(neighs)), neighs]).T
    [newf.write(f"{e[0]},{e[1]}\n") for e in edges]
    [totf.write(f"{e[0]},{e[1]}\n") for e in edges]
    
    searchstack.extend(neighs[~np.isin(neighs, explored)]) # get rid of already visited neighs, add rest to searchstack

explored sagalx, found 50 new users, 1 totally new, 59345 total
explored classicalbird, found 50 new users, 0 totally new, 59345 total
explored Arthur_Zeitter, found 50 new users, 0 totally new, 59345 total
explored Ykswo, found 50 new users, 0 totally new, 59345 total
explored sandundermyfeet, found 50 new users, 0 totally new, 59345 total
explored YaBoiRockstar, found 50 new users, 0 totally new, 59345 total
explored whitech0c0late, found 50 new users, 0 totally new, 59345 total
explored Alessandro19ds, found 0 new users, 0 totally new, 59345 total


C:\Users\barbi\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


explored staywithziam, found 50 new users, 3 totally new, 59348 total
explored Juandamejiam, found 32 new users, 0 totally new, 59348 total
explored LouisThorne23, found 47 new users, 0 totally new, 59348 total
explored Hebastian15, found 25 new users, 0 totally new, 59348 total
explored IfUSeekLucas, found 48 new users, 0 totally new, 59348 total
explored olegoleg24, found 47 new users, 0 totally new, 59348 total
explored Interspeed177, found 37 new users, 0 totally new, 59348 total
explored sajan__, found 20 new users, 0 totally new, 59348 total
explored Sumi_Licious, found 13 new users, 0 totally new, 59348 total
explored leonardopeh189, found 5 new users, 0 totally new, 59348 total
explored savosuh10, found 4 new users, 0 totally new, 59348 total
explored Willycyrus23, found 35 new users, 0 totally new, 59348 total
explored lucgsas, found 28 new users, 0 totally new, 59348 total
explored mariadr3w, found 42 new users, 1 totally new, 59349 total
explored caleblg, found 6 new users, 

KeyboardInterrupt: 

In [15]:
totf.close()
newf.close()

## bashdu

In [108]:
year = pd.read_csv("./d2/listens_year_10k100.csv")
allt = pd.read_csv("./d2/listens_10k200.csv")

In [112]:
allt[["user_id", "artist_id", "scrobbles"]].to_csv("./d2/lastfm_user_scrobbles_10k200_through2021.csv")